# Reconciliation - Jypyter Notebook Example
This notebook aims to provide a quick and easy example of how to develop and deploy a reconciliation application using python's Jupyter Notebook environment

### User Inputs
* System1 data
* System2 data
* Row mappings to link system1 and system2 data
* Column mappings to extract and map data from each system

### Output
* Excel spreadsheet with reconciled data and inputs
* Database with data stored in tables

In [44]:
from rekon import rekon
import pandas as pd
import zipfile

## Use pandas to read excel file

In [4]:
# lets get sample data from excel file
excel_file = '/Users/rjdscott/Desktop/rekon-sample-data.xlsx'

# import the row and col mappings
col_map = pd.read_excel(excel_file,sheet_name='column_mapping', index_col=None)
row_map = pd.read_excel(excel_file,sheet_name='row_mapping', index_col=None)

# pull out required data from sheets based on col_map
sys1_df = pd.read_excel(excel_file, sheet_name='system1', index_col=None)
sys2_df = pd.read_excel(excel_file, sheet_name='system2', index_col=None)

# lets strip the white spaces from items
sys1_df = sys1_df.applymap(lambda x: x.strip() if type(x) is str else x)
sys2_df = sys2_df.applymap(lambda x: x.strip() if type(x) is str else x)
col_map = col_map.applymap(lambda x: x.strip() if type(x) is str else x)
row_map = row_map.applymap(lambda x: x.strip() if type(x) is str else x)

In [5]:
sys1_df.head()

,Ticker,TotalUnitHolding,MarketPrice,TotalValueBase
0,BABA US,5,178.19,1212.92
1,ANZ,12,28.78,345.36
2,BHP,20,33.38,667.60
3,BLD,36,6.42,231.12
4,BRBY LN,32,2325.00,1304.62


In [6]:
sys2_df.head() 

,SecurityCode,PriceMultiplier,PositionVolume,Cost,CostBase,CostPrice,NativeCostPrice,AverageFXRate,LastPrice,MarketPrice,MarketValue,NativeMarketValue
0,ANZ.ASX,0.01,12,351.14,351.14,2926.166667,2926.166667,1.000000,2943.0,2943.000000,353.16,353.16
1,BABA.NYS,1.00,5,1291.56,1291.56,258.312000,191.912000,0.742947,178.5,244.196068,1220.98,892.50
2,BHP.ASX,0.01,20,682.64,682.64,3413.200000,3413.200000,1.000000,3381.0,3381.000000,676.20,676.20
3,BLD.ASX,0.01,36,233.82,233.82,649.500000,649.500000,1.000000,707.0,707.000000,254.52,254.52
4,BRBY.LSE,0.01,32,1284.46,1284.46,4013.937500,2300.187500,0.573050,2310.0,4117.280100,1317.53,739.20


In [7]:
row_map.head()

,system1,system2
0,BABA US,BABA.NYS
1,ANZ,ANZ.ASX
2,BHP,BHP.ASX
3,BLD,BLD.ASX
4,BRBY LN,BRBY.LSE


In [8]:
col_map.head()

,system1,system2
0,Ticker,SecurityCode
1,MarketPrice,MarketPrice
2,TotalUnitHolding,PositionVolume
3,TotalValueBase,MarketValue


## Perform reconciliation

In [24]:
# create reconciliation class
r = rekon.Reconciliation(system1_data=sys1_df, 
                         system2_data=sys2_df,
                         column_mapping=col_map,
                         row_mappings=row_map,
                         system_labels=row_map.columns)

# call reconcile class method using sqlite in memory, reconciling col1 
r.reconcile(rec_column=3, sqlite_db=":memory:")

# need to fix the pretty outputer

Normalising tables...
Normalised tables inserted to database...
Mappings added to database..


,s1_key,s2_key,s1_value3,s2_value3,s1s2_value3_diff
0,BABA US,BABA.NYS,1212.92,1220.98,-8.06
1,ANZ,ANZ.ASX,345.36,353.16,-7.80
2,BHP,BHP.ASX,667.60,676.20,-8.60
3,BLD,BLD.ASX,231.12,254.52,-23.40
4,BRBY LN,BRBY.LSE,1304.62,1317.53,-12.91
5,CBA,CBA.ASX,143.46,146.46,-3.00
6,IVV,IVV.AXW,797.00,800.42,-3.42
7,LKE,LKE.ASX,890.00,890.00,0.00
8,NAB,NAB.ASX,307.78,313.28,-5.50
9,TLS,TLS.ASX,241.68,238.64,3.04


In [43]:
# output files to excel
writer = pd.ExcelWriter('example-excel.xlsx')
r.rec_result_pretty.to_excel(writer, sheet_name='rec_report',index=False)
r.system1_data.to_excel(writer, sheet_name='system1_data',index=False)
r.system2_data.to_excel(writer, sheet_name='system2_data',index=False)
writer.save()

In [46]:
# output files to csv and zip
with zipfile.ZipFile('/Users/rjdscott/Desktop/test_zipfile.zip', 'w') as csv_zip:
    csv_zip.writestr("rec_result.csv", r.rec_result_pretty.to_csv())
    csv_zip.writestr("system1_data.csv", r.system1_data.to_csv())
    csv_zip.writestr("system2_data.csv", r.system2_data.to_csv())

In [ ]:
# output csv's to new directory
# not working for now

In [26]:
df = r.rec_result_pretty
print('number of rows:   {}'.format(len(df)))
print("number of breaks: {}".format(df['diff'].astype(bool).sum(axis=0)))

r.rec_result_pretty['diff'].astype(bool).sum(axis=0)

number of rows:   13
number of breaks: 12


12

In [30]:
def output_stats(df):
    num_rows = len(df)
    num_breaks = df['diff'].astype(bool).sum(axis=0)
    left_misses = df.iloc[:,0].isna().sum()
    right_misses = df.iloc[:,1].isna().sum()
    print('num rows:{}'.format(num_rows))
    print('num breaks:{}'.format(num_rows))
    print('left misses:{}'.format(left_misses))
    print('right_misses:{}'.format(right_misses))

          
r2 = rekon.Reconciliation()
r2.load_sample_data()
r2.reconcile()

output_stats(r2.rec_result_pretty)

['system1', 'system2']
    system1 system2
0   product    item
1  quantity  volume
2     price    cost
  system1 system2
0       a      a1
1       b      b1
2       c      c1
3       d      d1
  product  quantity  price
0       a       100      1
1       b        22      5
2       c        10     10
3       d        20      3
  item  volume  cost
0   a1     100     1
1   b1      20     5
2   c1      10    10
Normalising tables...
Normalised tables inserted to database...
Mappings added to database..
num rows:4
num breaks:4
left misses:0
right_misses:1
